In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from layers.RegexTokenizer import RegexTokenizer
import torch
from torch import nn
from torch.nn import functional as F
import time

from utils import ModelSpecs, TrainingData
# from layers.FeedForward import FeedFoward
# from layers.MultiHeadAttention import MultiHeadAttention
from layers.Block import Block
torch.manual_seed(1337)

In [20]:
modelSpecs = ModelSpecs.create('toy')

In [21]:
# hyperparameters
BATCH_SIZE = 64 # how many independent sequences will we process in parallel?
# BLOCK_SIZE = 256 # what is the maximum context length for predictions?
MAX_ITERS = 2000
EVAL_INTERVALS = 100
LEARNING_RATE = 3e-4
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
# EVAL_ITERS = 200
EVAL_ITERS = 50
# N_EMBD = 884
# N_EMBD = 10
# N_HEAD = 6
# N_LAYER = 6
# DROPOUT = 0.2

TRAIN_TEST_SPLIT = 0.9
# VOCAB_SIZE = 1800

# MODEL_SPECS = { 
#     "VOCAB_SIZE" : 1800,
#     "BLOCK_SIZE" : BLOCK_SIZE, 
#     "N_EMBD" : 384, 
#     "N_HEAD" : 6, 
#     "N_LAYER" : 6, 
#     "DROPOUT" : 0.2, 
# }

print(f"Running on '{device.capitalize()}'device.")


Running on 'Cpu'device.


In [22]:
Tokenizer = RegexTokenizer()
# text = TrainingData.stories()
text = TrainingData.TinyStories()[:30000]

try :
    Tokenizer.load(f"weights/stories{modelSpecs.VOCAB_SIZE}.model")
except :
    Tokenizer.train(text, modelSpecs.VOCAB_SIZE, True)
    Tokenizer.save(f"weights/stories{modelSpecs.VOCAB_SIZE}")

data = torch.tensor(Tokenizer.encode(text))

n = int(TRAIN_TEST_SPLIT * len(data))
print(len(data))

train_data = data[:n]
val_data = data[n:]


11821


In [23]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - modelSpecs.BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+modelSpecs.BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+modelSpecs.BLOCK_SIZE+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [24]:
@torch.no_grad()
def estimate_loss(model : nn.Module):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [25]:
# class Head(nn.Module):
#     """ one head of self-attention """

#     def __init__(self, head_size, specs: ModelSpecs):
#         super().__init__()
#         n_embd = specs.N_EMBD
#         block_size = specs.BLOCK_SIZE
#         self.key = nn.Linear(n_embd, head_size, bias=False)
#         self.query = nn.Linear(n_embd, head_size, bias=False)
#         self.value = nn.Linear(n_embd, head_size, bias=False)
#         self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

#         self.dropout = nn.Dropout(DROPOUT)

#     def forward(self, x : torch.Tensor):
#         # input of size (batch, time-step, channels)
#         # output of size (batch, time-step, head size)
#         B,T,C = x.shape
#         k : torch.Tensor= self.key(x)   # (B,T,hs)
#         q = self.query(x) # (B,T,hs)
#         # compute attention scores ("affinities")
#         wei :torch.Tensor = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
#         wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
#         wei = F.softmax(wei, dim=-1) # (B, T, T)
#         wei = self.dropout(wei)
#         # perform the weighted aggregation of the values
#         v = self.value(x) # (B,T,hs)
#         out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
#         return out


In [26]:
# class MultiHeadAttention(nn.Module):
#     """ multiple heads of self-attention in parallel """

#     def __init__(self, num_heads, head_size):
#         super().__init__()
#         self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
#         self.proj = nn.Linear(head_size * num_heads, N_EMBD)
#         self.dropout = nn.Dropout(DROPOUT)

#     def forward(self, x):
#         out = torch.cat([h(x) for h in self.heads], dim=-1)
#         out = self.dropout(self.proj(out))
#         return out


In [27]:
# class FeedFoward(nn.Module):
#     """ a simple linear layer followed by a non-linearity """

#     def __init__(self, n_embd):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(n_embd, 4 * n_embd),
#             nn.ReLU(),
#             nn.Linear(4 * n_embd, n_embd),
#             nn.Dropout(DROPOUT),
#         )

#     def forward(self, x):
#         return self.net(x)


In [28]:
# class Block(nn.Module):
#     """ Transformer block: communication followed by computation """

#     def __init__(self, n_embd, n_head):
#         # n_embd: embedding dimension, n_head: the number of heads we'd like
#         super().__init__()
#         head_size = n_embd // n_head
#         self.sa = MultiHeadAttention(n_head, head_size)
#         self.ffwd = FeedFoward(n_embd)
#         self.ln1 = nn.LayerNorm(n_embd)
#         self.ln2 = nn.LayerNorm(n_embd)

#     def forward(self, x):
#         x = x + self.sa(self.ln1(x))
#         x = x + self.ffwd(self.ln2(x))
#         return x


In [29]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(modelSpecs.VOCAB_SIZE, modelSpecs.N_EMBD)
        self.position_embedding_table = nn.Embedding(modelSpecs.BLOCK_SIZE, modelSpecs.N_EMBD)
        self.blocks = nn.Sequential(*[Block(modelSpecs) for _ in range(modelSpecs.N_LAYER)])
        self.ln_f = nn.LayerNorm(modelSpecs.N_EMBD) # final layer norm
        self.lm_head = nn.Linear(modelSpecs.N_EMBD, modelSpecs.VOCAB_SIZE)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -modelSpecs.BLOCK_SIZE:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [30]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)


0.730612 M parameters


In [ ]:
startTime = time.time()

for iter in range(MAX_ITERS):
    print(f"iter #{iter}")
    torch.cuda.empty_cache()
    # every once in a while evaluate the loss on train and val sets
    if iter % EVAL_INTERVALS == 0 or iter == MAX_ITERS - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time {int((time.time() - startTime)//60)} minutes")
    torch.cuda.empty_cache()
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

endTime = time.time()
print(f"Total Training Time : {int((endTime - startTime)//60)} minutes")

iter #0
step 0: train loss 6.2642, val loss 6.2578, time 0 minutes
iter #1
iter #2
iter #3
iter #4
iter #5
iter #6
iter #7
iter #8
iter #9
iter #10
iter #11
iter #12
iter #13
iter #14
iter #15
iter #16
iter #17
iter #18
iter #19
iter #20
iter #21
iter #22
iter #23
iter #24
iter #25
iter #26
iter #27
iter #28
iter #29
iter #30
iter #31
iter #32
iter #33
iter #34
iter #35
iter #36
iter #37
iter #38
iter #39
iter #40
iter #41
iter #42
iter #43
iter #44
iter #45
iter #46
iter #47
iter #48
iter #49
iter #50
iter #51
iter #52
iter #53
iter #54
iter #55
iter #56
iter #57
iter #58
iter #59
iter #60
iter #61
iter #62
iter #63
iter #64
iter #65
iter #66
iter #67
iter #68
iter #69
iter #70
iter #71
iter #72
iter #73
iter #74
iter #75
iter #76
iter #77
iter #78
iter #79
iter #80
iter #81
iter #82
iter #83
iter #84
iter #85
iter #86
iter #87
iter #88
iter #89
iter #90
iter #91
iter #92
iter #93
iter #94
iter #95
iter #96
iter #97
iter #98
iter #99
iter #100
step 100: train loss 3.9227, val loss 4.3

In [ ]:
# generate from the model
text = "hi, my name is Arush. following is my introduction."
m.eval()
context = torch.tensor(Tokenizer.encode(text), dtype=torch.long, device=device).unsqueeze(0)
# context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(Tokenizer.decode(m.generate(context, max_new_tokens=32)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))
m.train()

hi, my name is Arush. following is my introduction. We are
domybrogistence in the chairdestic disgention of
incing—ascrious b
